# Numerical Example for Proximal of the Max Norm

In [97]:
import numpy as np

def get_root(x, lambd, tol=1.0e-10):
    ''' Execute the bisection method to find parameter for L1 ball projection
    '''
    t_lo = 0.0
    t_hi = np.max(np.abs(x))

    converge = False
    while not converge:
        t = 0.5 * (t_lo + t_hi)
        res = np.sum(np.maximum(np.abs(x) - t, 0)) - lambd
        if res > 0:
            t_lo = t
        else:
            t_hi = t
        converge = np.abs(res) < tol * lambd
    return t


def prox_max_norm(x, lambd):
    ''' Compute proximal for max norm using Moreau identity
    '''
    l1_norm = np.sum(np.abs(x))
    if l1_norm <= lambd:
        return np.zeros(x.shape)
    else:
        alpha = get_root(x, lambd)
        prox = x - np.sign(x) * np.maximum(np.abs(x) - alpha, 0)
        return prox


def proj_rank(A, r):
    ''' Projection matrix onto set of rank r matrices
    '''
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S[r:] = 0
    return (U * S) @ V


def T(Z, lambd, r, ord='max'):
    ''' Apply fixed point operator to minimize |X - D|_∞ over rank r matrices

        Note: This is a Douglas Rachford update, and the iteration is not
              convex. Thus, it will only converge for step size lambd that is
              sufficiently small.
    '''
    X = proj_rank(Z, r)
    Z = Z + prox_max_norm(2 * X - Z - D, lambd) + D - X
    return X, Z


def compute_low_rank_est(D, r, lambd=1e-3, ord='fro', iters=500):
    ''' Compute low-rank estimate of D w.r.t. the max norm
    '''
    err_max = np.zeros(iters)
    err_fro = np.zeros(iters)
    rank    = np.zeros(iters)
    norm    = np.zeros(iters)
    Z       = D.copy()

    for k in range(iters):
        Z_p  = Z.copy()
        X, Z = T(Z, lambd, r, ord=ord)

        err_max[k] = np.max(np.abs(X - D)) / np.max(np.abs(D))
        err_fro[k] = np.linalg.norm(X - D, ord='fro') / np.linalg.norm(D, ord='fro')
        rank[k]    = np.linalg.matrix_rank(X)
        norm[k]    = np.linalg.norm(X, ord='nuc')

        if k % 10 == 0:
            msg  = "[{:5d}]: |Z - Z_p| = {:0.3e}, fro err = {:0.3e}, "
            msg += "max err = {:0.3e}, |X|_* = {:0.3e}, rank = {:2.0f}"
            print(msg.format(k, np.linalg.norm(Z - Z_p, ord='fro'),
                             err_fro[k], err_max[k], norm[k], rank[k]))

    return X, err_max, err_fro, rank, norm


## Convex Example

We use the fixed point iteration acceleration from this [paper](https://arxiv.org/pdf/2206.09462)

In [ ]:
np.random.seed(2024)
m = 500
n = 1000
r = 10

# Create a Gaussian matrix D with decaying singular values
D = np.random.normal(0, 1.0, size=(m,n))
U, S, V = np.linalg.svd(D, full_matrices=False)
S = np.exp(-0.5 * np.arange(S.size))
D = (U * S) @ V

X_max, err_max, err_fro, rank, norm = compute_low_rank_est(D, r)

[    0]: |Z - Z_p| = 8.466e-03, fro err = 6.738e-03, max err = 5.740e-03, |X|_* = 2.524e+00, rank = 10
[   10]: |Z - Z_p| = 3.827e-05, fro err = 6.742e-03, max err = 5.121e-03, |X|_* = 2.524e+00, rank = 10
[   20]: |Z - Z_p| = 3.212e-05, fro err = 6.752e-03, max err = 4.887e-03, |X|_* = 2.524e+00, rank = 10
[   30]: |Z - Z_p| = 2.777e-05, fro err = 6.758e-03, max err = 4.708e-03, |X|_* = 2.524e+00, rank = 10
[   40]: |Z - Z_p| = 2.436e-05, fro err = 6.761e-03, max err = 4.582e-03, |X|_* = 2.524e+00, rank = 10
[   50]: |Z - Z_p| = 2.145e-05, fro err = 6.763e-03, max err = 4.460e-03, |X|_* = 2.524e+00, rank = 10
[   60]: |Z - Z_p| = 1.832e-05, fro err = 6.766e-03, max err = 4.407e-03, |X|_* = 2.524e+00, rank = 10
[   70]: |Z - Z_p| = 1.526e-05, fro err = 6.768e-03, max err = 4.323e-03, |X|_* = 2.524e+00, rank = 10
[   80]: |Z - Z_p| = 1.233e-05, fro err = 6.771e-03, max err = 4.305e-03, |X|_* = 2.524e+00, rank = 10
[   90]: |Z - Z_p| = 1.087e-05, fro err = 6.774e-03, max err = 4.230e-03,

Lastly, we compare the errors of using the above formulation versus simply projecting $D$ onto the set of rank 10 matrices.

In [100]:
X_fro = proj_rank(D, 10)

print('|X_max - D|_max / |D|_max =', np.max(np.abs(X_max - D)) / np.max(np.abs(D)))
print('|X_max - D|_F   / |D|_F   =',  np.linalg.norm(X_max - D, ord='fro') / np.linalg.norm(D, ord='fro'))
print('rank(X_max) = ', np.linalg.matrix_rank(X_max), '\n')

print('|X_fro - D|_max / |D|_max =', np.max(np.abs(X_fro - D) / np.max(np.abs(D))))
print('|X_fro - D|_F   / |D|_F   =',  np.linalg.norm(X_fro - D, ord='fro') / np.linalg.norm(D, ord='fro'))
print('rank(X_fro) = ', np.linalg.matrix_rank(X_fro))

|X_max - D|_max / |D|_max = 0.003986367427823933
|X_max - D|_F   / |D|_F   = 0.006831167104625575
rank(X_max) =  10 

|X_fro - D|_max / |D|_max = 0.005739844395175539
|X_fro - D|_F   / |D|_F   = 0.006737946999085468
rank(X_fro) =  10
